# Legal document classification in zero-shot cross lingual transfer setting

# Part III: Performance improvement and pattern analysis

Date: May 2025

Project of course: Natural Language Processing - ENSAE 3A S2

Author: Noémie Guibé

In [2]:
# import 
import pandas as pd
import spacy
from tqdm import tqdm
from collections import defaultdict, Counter
import itertools
import matplotlib.pyplot as plt

In [5]:
python3 -m spacy download en_core_web_sm

SyntaxError: invalid syntax (2180754472.py, line 1)

In [7]:
!python -m spacy download en_core_web_sm
!python -m spacy download fr_core_news_sm
!python -m spacy download de_core_news_sm
!python -m spacy download pl_core_news_sm
!python -m spacy download fi_core_news_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.1/12.8 MB 2.2 MB/s eta 0:00:06
      --------------------------------------- 0.2/12.8 MB 2.0 MB/s eta 0:00:07
     - -------------------------------------- 0.3/12.8 MB 2.0 MB/s eta 0:00:07
     - -------------------------------------- 0.5/12.8 MB 2.2 MB/s eta 0:00:06
     - -------------------------------------- 0.6/12.8 MB 2.5 MB/s eta 0:00:05
     -- ------------------------------------- 0.7/12.8 MB 2.4 MB/s eta 0:00:06
     -- ------------------------------------- 0.7/12.8 MB 2.4 MB/s eta 0:00:06
     -- ------------------------------------- 0.9/12.8 MB 2.2 MB/s eta 0:00:06
     --- ------------------------------------ 1.2/12.8 MB 2.7 MB/s eta 0:00:05
     ---- ----------------------------------- 1.4/12.8 MB 2.8 MB/s eta 0:00:05
     ----- ---------------------------------- 1.7/12.8 MB 3.1 MB/s eta 0:00:04
     ------ --------------------------------- 2.0/12.8 MB 3

# 1 - Original model through token analysis

## Cleaning and lemmatization

In [ ]:
# Load spaCy models for each language
spacy_models = {
    "en": spacy.load("en_core_web_sm"),
    "fr": spacy.load("fr_core_news_sm"),
    "de": spacy.load("de_core_news_sm"),
    "pl": spacy.load("pl_core_news_sm"),
    "fi": spacy.load("fi_core_news_sm")
}

# List of languages you care about
languages = ["en", "fr", "de", "pl", "fi"]

# Function to clean and lemmatize text
def clean_and_lemmatize(text, lang_code, remove_stopwords=True):
    if lang_code not in spacy_models:
        return None
    
    nlp = spacy_models[lang_code]
    doc = nlp(text)

    tokens = [
        token.lemma_.lower() for token in doc
        if not token.is_punct and not token.is_space and (not token.is_stop if remove_stopwords else True)
    ]
    return " ".join(tokens)


In [ ]:
# Apply cleaning/lemmatization across the dataframe
def process_dataframe(df, languages):
    lemmatized_texts = []

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        text_dict = row["text"]  # {lang: text}
        labels = row["level_1_labels"]

        for lang in languages:
            if isinstance(text_dict, dict) and lang in text_dict:
                raw_text = text_dict[lang]
                lemmatized = clean_and_lemmatize(raw_text, lang)
                if lemmatized:
                    lemmatized_texts.append({
                        "lang": lang,
                        "text_lemmatized": lemmatized,
                        "labels": labels
                    })

    return pd.DataFrame(lemmatized_texts)



In [ ]:
# Usage
df = pd.read_parquet("data/dataset/multi_eurlex_reduced.parquet")
processed_df = process_dataframe(df, languages)

# Save for later use
processed_df.to_parquet("data/processed_legal_texts.parquet", index=False)


## Token - Label co-occurence analysis

In [ ]:
# Load preprocessed lemmatized texts
df = pd.read_parquet("data/processed_legal_texts.parquet")

# Build token-label co-occurrence mapping
token_label_counts = defaultdict(Counter)

for _, row in df.iterrows():
    tokens = row["text_lemmatized"].split()
    labels = row["labels"]
    
# Count each token against all its labels
    for token in set(tokens):
        for label in labels:
            token_label_counts[token][label] += 1

# Convert to DataFrame for inspection
top_tokens = sorted(token_label_counts.items(), key=lambda x: sum(x[1].values()), reverse=True)[:100]

rows = []
for token, label_counter in top_tokens:
    for label, count in label_counter.items():
        rows.append({"token": token, "label": label, "count": count})

df_token_label = pd.DataFrame(rows)
df_token_label.to_csv("output/token_label_cooccurrence.csv", index=False)


In [ ]:
# Optional: Visualize token-label counts for one label
def plot_top_tokens_for_label(label, top_n=10):
    label_filtered = df_token_label[df_token_label["label"] == label]
    top = label_filtered.sort_values("count", ascending=False).head(top_n)
    plt.figure(figsize=(10, 5))
    plt.bar(top["token"], top["count"])
    plt.title(f"Top {top_n} tokens for label {label}")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


In [ ]:
# Example: plot_top_tokens_for_label("0806")

## Token distribution accross languages

In [ ]:
# Load the same preprocessed file
df = pd.read_parquet("data/processed_legal_texts.parquet")

# Count token frequencies by language
lang_token_freq = defaultdict(Counter)

for _, row in df.iterrows():
    lang = row["lang"]
    tokens = row["text_lemmatized"].split()
    lang_token_freq[lang].update(tokens)

# Create a DataFrame with token frequencies across languages
def get_freq_df(top_tokens=None, min_freq=50):
    all_tokens = set()
    if top_tokens:
        all_tokens = set(top_tokens)
    else:
        # Get common tokens across languages
        for lang, counter in lang_token_freq.items():
            common = {token for token, freq in counter.items() if freq > min_freq}
            all_tokens |= common

    data = []
    for token in all_tokens:
        row = {"token": token}
        for lang in lang_token_freq:
            row[lang] = lang_token_freq[lang][token]
        data.append(row)
    
    return pd.DataFrame(data)


In [ ]:
# Plot distribution of selected tokens
def plot_token_distribution(tokens):
    df_freq = get_freq_df(tokens)
    df_freq.set_index("token").T.plot(kind='bar', figsize=(10, 5))
    plt.title("Token frequency across languages")
    plt.ylabel("Frequency")
    plt.xlabel("Language")
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()


In [ ]:
# Example usage:
# plot_token_distribution(["regulation", "union", "market", "recht", "union", "protection"])


What You Can Explore With These:
Token–Label Co-occurrence:
•	Find tokens that strongly signal a specific legal topic (label).
•	Detect domain-specific terms frequently co-occurring with multi-label groups.
Cross-Language Token Comparison:
•	Discover shared Latin-root legal vocabulary (e.g., regulation, directive, protection).
•	Identify language-specific phrasing or underrepresentation of concepts.



# 2 - Other strategies